In [0]:
dbutils.library.installPyPI("fuzzywuzzy")
# dbutils.library.installPyPI("Keras")
# dbutils.library.installPyPI("nltk")
# dbutils.library.installPyPI("numpy")

Out[1]: True

In [0]:
! /databricks/python/bin/pip install --upgrade pip #pip install googletrans
! /databricks/python/bin/pip install nltk
! /databricks/python/bin/python -m nltk.downloader stopwords
! /databricks/python/bin/python -m nltk.downloader punkt
! /databricks/python/bin/python -m nltk.downloader wordnet

In [0]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import nltk
from pyspark.sql.functions import *
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import re
from keras.preprocessing.text import Tokenizer
import tensorflow as tf

/local_disk0/pythonVirtualEnvDirs/virtualEnv-f09e8ddf-eb18-40ad-b257-791b939882a5/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
 warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')

## Bulding a text summarizer to summerize text messages to keep the most important stuff in the chat and discard the irrelevant stuff as it will not add any meaning.

### Proof of Concept for Text Summarizer

In [0]:
# Input text - to summarize  
text = """There are many techniques available to generate extractive summarization to keep it simple, I will be using an unsupervised learning approach to find the sentences similarity and rank them. Summarization can be defined as a task of producing a concise and fluent summary while preserving key information and overall meaning. One benefit of this will be, you don’t need to train and build a model prior start using it for your project. It’s good to understand Cosine similarity to make the best use of the code you are going to see. Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. Its measures cosine of the angle between vectors. The angle will be 0 if sentences are similar."""
   

In [0]:
# Tokenizing the text 
stopWords = set(stopwords.words("english")) 
words = word_tokenize(text)
words

Out[5]: ['There',
 'are',
 'many',
 'techniques',
 'available',
 'to',
 'generate',
 'extractive',
 'summarization',
 'to',
 'keep',
 'it',
 'simple',
 ',',
 'I',
 'will',
 'be',
 'using',
 'an',
 'unsupervised',
 'learning',
 'approach',
 'to',
 'find',
 'the',
 'sentences',
 'similarity',
 'and',
 'rank',
 'them',
 '.',
 'Summarization',
 'can',
 'be',
 'defined',
 'as',
 'a',
 'task',
 'of',
 'producing',
 'a',
 'concise',
 'and',
 'fluent',
 'summary',
 'while',
 'preserving',
 'key',
 'information',
 'and',
 'overall',
 'meaning',
 '.',
 'One',
 'benefit',
 'of',
 'this',
 'will',
 'be',
 ',',
 'you',
 'don',
 '’',
 't',
 'need',
 'to',
 'train',
 'and',
 'build',
 'a',
 'model',
 'prior',
 'start',
 'using',
 'it',
 'for',
 'your',
 'project',
 '.',
 'It',
 '’',
 's',
 'good',
 'to',
 'understand',
 'Cosine',
 'similarity',
 'to',
 'make',
 'the',
 'best',
 'use',
 'of',
 'the',
 'code',
 'you',
 'are',
 'going',
 'to',
 'see',
 '.',
 'Cosine',
 'similarity',
 'is',
 'a',
 'measure',
 'of',
 'similarity',
 'between',
 'two',
 'non-zero',
 'vectors',
 'of',
 'an',
 'inner',
 'product',
 'space',
 'that',
 'measures',
 'the',
 'cosine',
 'of',
 'the',
 'angle',
 'between',
 'them',
 '.',
 'Its',
 'measures',
 'cosine',
 'of',
 'the',
 'angle',
 'between',
 'vectors',
 '.',
 'The',
 'angle',
 'will',
 'be',
 '0',
 'if',
 'sentences',
 'are',
 'similar',
 '.']

In [0]:
sentences = sent_tokenize(text) 
sentences

Out[6]: ['There are many techniques available to generate extractive summarization to keep it simple, I will be using an unsupervised learning approach to find the sentences similarity and rank them.',
 'Summarization can be defined as a task of producing a concise and fluent summary while preserving key information and overall meaning.',
 'One benefit of this will be, you don’t need to train and build a model prior start using it for your project.',
 'It’s good to understand Cosine similarity to make the best use of the code you are going to see.',
 'Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them.',
 'Its measures cosine of the angle between vectors.',
 'The angle will be 0 if sentences are similar.']

In [0]:
# Creating a frequency table to keep the  
# score of each word 
freqTable = dict() 
for word in words: 
    word = word.lower() 
    if word in stopWords: 
        continue
    if word in freqTable: 
        freqTable[word] += 1
    else: 
        freqTable[word] = 1

In [0]:
freqTable

Out[8]: {'many': 1,
 'techniques': 1,
 'available': 1,
 'generate': 1,
 'extractive': 1,
 'summarization': 2,
 'keep': 1,
 'simple': 1,
 ',': 2,
 'using': 2,
 'unsupervised': 1,
 'learning': 1,
 'approach': 1,
 'find': 1,
 'sentences': 2,
 'similarity': 4,
 'rank': 1,
 '.': 7,
 'defined': 1,
 'task': 1,
 'producing': 1,
 'concise': 1,
 'fluent': 1,
 'summary': 1,
 'preserving': 1,
 'key': 1,
 'information': 1,
 'overall': 1,
 'meaning': 1,
 'one': 1,
 'benefit': 1,
 '’': 2,
 'need': 1,
 'train': 1,
 'build': 1,
 'model': 1,
 'prior': 1,
 'start': 1,
 'project': 1,
 'good': 1,
 'understand': 1,
 'cosine': 4,
 'make': 1,
 'best': 1,
 'use': 1,
 'code': 1,
 'going': 1,
 'see': 1,
 'measure': 1,
 'two': 1,
 'non-zero': 1,
 'vectors': 2,
 'inner': 1,
 'product': 1,
 'space': 1,
 'measures': 2,
 'angle': 3,
 '0': 1,
 'similar': 1}

In [0]:
len(freqTable)

Out[9]: 59

In [0]:
# Creating a dictionary to keep the score 
# of each sentence 
sentences = sent_tokenize(text) 
sentenceValue = dict() 
   
for sentence in sentences: 
    for word, freq in freqTable.items(): 
        if word in sentence.lower(): 
            if sentence in sentenceValue: 
                sentenceValue[sentence] += freq 
            else: 
                sentenceValue[sentence] = freq 

In [0]:
sentenceValue

Out[11]: {'There are many techniques available to generate extractive summarization to keep it simple, I will be using an unsupervised learning approach to find the sentences similarity and rank them.': 32,
 'Summarization can be defined as a task of producing a concise and fluent summary while preserving key information and overall meaning.': 20,
 'One benefit of this will be, you don’t need to train and build a model prior start using it for your project.': 22,
 'It’s good to understand Cosine similarity to make the best use of the code you are going to see.': 26,
 'Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them.': 29,
 'Its measures cosine of the angle between vectors.': 19,
 'The angle will be 0 if sentences are similar.': 14}

In [0]:
len(sentenceValue)

Out[12]: 7

In [0]:
sumValues = 0
for sentence in sentenceValue: 
    sumValues += sentenceValue[sentence] 

In [0]:
sumValues

Out[14]: 162

In [0]:
# Average value of a sentence from the original text 
average = int(sumValues / len(sentenceValue)) 
average

Out[15]: 23

In [0]:
1.2 * average

Out[16]: 27.599999999999998

In [0]:
# Storing sentences into our summary. 
summary = '' 
for sentence in sentences: 
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)): 
        summary += " " + sentence 
print(summary) 

There are many techniques available to generate extractive summarization to keep it simple, I will be using an unsupervised learning approach to find the sentences similarity and rank them. Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them.

In [0]:
def chat_summerizer(text):
  # Tokenizing the text 
  stopWords = set(stopwords.words("english")) 
  words = word_tokenize(text)
  sentences = sent_tokenize(text) 
  
  # Creating a frequency table to keep the score of each word 
  freqTable = dict() 
  for word in words: 
      word = word.lower() 
      if word in stopWords: 
          continue
      if word in freqTable: 
          freqTable[word] += 1
      else: 
          freqTable[word] = 1

  # Creating a dictionary to keep the score of each sentence 
  sentences = sent_tokenize(text) 
  sentenceValue = dict() 
   
  for sentence in sentences: 
      for word, freq in freqTable.items(): 
          if word in sentence.lower(): 
              if sentence in sentenceValue: 
                  sentenceValue[sentence] += freq 
              else: 
                  sentenceValue[sentence] = freq 
                  
  sumValues = 0
  for sentence in sentenceValue: 
      sumValues += sentenceValue[sentence] 
      
  # Average value of a sentence from the original text 
  average = int(sumValues / len(sentenceValue)) 
  
  # Storing sentences into our summary
  summary = '' 
  for sentence in sentences: 
      if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)): 
          summary += " " + sentence 
  # print(summary)
  return summary

In [0]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text
  
def lemmatize_text(text):
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]

def pre_processing(df,col_name):
  df['cleaned_text'] = df[col_name].apply(lambda x: (clean_text(x)))
  df['cleaned_text'] = df['cleaned_text'].str.replace('[^\w\s]','')
  stop = stopwords.words('english')
  df['stopwords_removed'] = df['cleaned_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  df['text_lemmatized'] = df.stopwords_removed.apply(lemmatize_text)
  df['text_lemmatized']=df['text_lemmatized'].apply(lambda x: " ".join(a for a in x))
  # df['text_lemmatized'] = df['text_lemmatized'].str.replace('[^\w\s]','')
  return df

def prep_train_data():
  df_types = spark.table("irumdb.cs_types_list")
  df_types=df_types.filter(df_types.Questions.isNotNull())
  df_types=df_types.na.replace(['14 Day trial '], [None], 'Questions')
  df_types=df_types.filter(df_types.Questions.isNotNull())
  
  df_types_pd = df_types.toPandas()
  final_df = pre_processing(df_types_pd,col_name='Questions')
  return final_df
  
def prep_chat_data():
  df_chats= spark.table("live_admin_chat.chatmessages")
  df_chats_pd=df_chats.toPandas()
  
  # TAKING ONLY FIRST 100 CHATS FOR CHECKING THE CONCEPT
  new_df=df_chats_pd[["ChatId","MessageText","MessageBy"]][100:200].groupby(["ChatId","MessageBy"],as_index=False).aggregate(lambda x: list(x))
  new_df['messages_split']=new_df['MessageText'].apply(lambda x: " ".join(a for a in x))
  new_df=new_df[~(new_df.messages_split.str.contains('Hi, welcome to the ENTERTAINER Live Chat'))]
  new_df=new_df[~(new_df.messages_split.str.contains('Your patience is much appreciated'))]
  new_df=new_df[~(new_df.messages_split.str.contains('NAVIGATEURL'))]
  new_df.reset_index(drop=True,inplace = True)
  
  new_df_sp = spark.createDataFrame(new_df)
  new_df_sp = new_df_sp.select('ChatId','messages_split').groupby('ChatId').agg(collect_set('messages_split')).sort(['ChatId'],ascending=True)
  
  new_df_pd=new_df_sp.toPandas()
  new_df_pd.rename(columns={"collect_set(messages_split)":"list_of_chats"},inplace=True)
  new_df_pd['list_of_chats']=new_df_pd['list_of_chats'].apply(lambda x: " ".join(a for a in x))
  
  new_df_pd['lower_chats'] = new_df_pd['list_of_chats'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  # FIRST TRY  
  # TEXT SUMMERIZER
  new_df_pd['summarized_text'] = new_df_pd['lower_chats'].apply(lambda x: (chat_summerizer(x)))
  
  stop = stopwords.words('english')
  # new_df_pd['stopwords_removed'] = new_df_pd['lower_chats'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  new_df_pd['stopwords_removed'] = new_df_pd['summarized_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  new_df_pd['stopwords_removed'] = new_df_pd['stopwords_removed'].str.replace('[^\w\s]','')
  freq=['nickchange','u','ok','okay','hi','thx','hello','entertainer','thanks','thankyou','thank you']
  new_df_pd['most_freq_removed'] = new_df_pd['stopwords_removed'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
  
  df_chats=pre_processing(new_df_pd,col_name='most_freq_removed')
  
#   # SECOND TRY TOTAL FLOP
#   # TEXT SUMMERIZER
#   df_chats['summarized_text'] = df_chats['most_freq_removed'].apply(lambda x: (chat_summerizer(x)))
  
  # df_chats=spark.createDataFrame(new_df_pd)
  # df_chats=df_chats.select('ChatId','most_freq_removed')
  return df_chats
  

In [0]:
df_quest = prep_train_data()
df_quest.head()

Out[10]:

,Categroy,Questions,cleaned_text,stopwords_removed,text_lemmatized
0,14 day trial,Can I try before I buy?,can i try before i buy,try buy,try buy
1,14 day trial,How can I try the offers?,how can i try the offers,try offers,try offer
2,14 day trial,How can I try the app?,how can i try the app,try app,try app
3,14 day trial,Can I try first?,can i try first,try first,try first
4,14 day trial,Can I try an offer before I buy?,can i try an offer before i buy,try offer buy,try offer buy


In [0]:
df_chats = prep_chat_data()
df_chats.head()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Did not pass numpy.dtype object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
Out[11]:

,ChatId,list_of_chats,lower_chats,summarized_text,stopwords_removed,most_freq_removed,cleaned_text,text_lemmatized
0,54137,Hi I spoke to Stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rec...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,speak stella earlier regard recent gourmet 202...
1,54141,Hello. This is my 3rd time on the chat I want ...,hello. this is my 3rd time on the chat i want ...,this is my 3rd time on the chat i want to cha...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnect e...
2,54143,"Hello there, Do i get repeated offers from sam...","hello there, do i get repeated offers from sam...",,,,,
3,54144,مرحبًا بك معنا مجددًا كان لديك 770 ابتسامة وا...,مرحبًا بك معنا مجددًا كان لديك 770 ابتسامة وان...,هل هناك أي شيء آخر يمكنني مساعدتك به ؟ [trans...,هل هناك أي شيء آخر يمكنني مساعدتك به transferc...,هل هناك أي شيء آخر يمكنني مساعدتك به transferc...,هل هناك أي شيء آخر يمكنني مساعدتك به transferc...,هل هناك أي شيء آخر يمكنني مساعدتك به transferc...
4,54145,Hi I was chatting with Greta How do I call you...,hi i was chatting with greta how do i call you...,hi i was chatting with greta how do i call yo...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chat greta call might easier purchase 2021 bun...


In [0]:
df_chats['text_lemmatized'].values

Out[12]: array(['speak stella earlier regard recent gourmet 2021 purchase want change purchase gourmet 2021 classic restaurants gourmet 2020 2021 anymore list restaurants less want refund gourmet 2021 doesnt much restaurants compare 2020 restaurants like anymore sandrheasacaresgmailcom sandrhea sacares sure yesi disappoint restaurants list gourmet 2020 2021 anymore yes know restaurants gourmet 2020 available classic 2021 see buy dubai gourmet 2021 today let know refund tend reach customers account within 10 work days',
 '3rd time chat want change package disconnect every single time ive try ask far ive tell wait everytime ive wait disconnect support person',
 '',
 'هل هناك أي شيء آخر يمكنني مساعدتك به transferchat transfer leslie amged عربي ar la مرحبا كان لدي سمايلز واختفت فجاة لماذا البريد ra11122211gmailcom الاسم roz abdullah تم ارتباط الحساب ولكن بدون سمايلز ليه ليه انتهت اوك تمام شكرا',
 'chat greta call might easier purchase 2021 bundle doesnt show need confirm purchase third time name harshika daryanani email address daryananiharshikagmailcom 0585504142 purchase 2021 bundle already 2020 key',
 '', 'jeeyeon ajycom1120gmailcom order number 10138751081861'],
 dtype=object)

In [0]:
# FIRST TRY WITH JUST SUMMERIZER
df_chats = prep_chat_data()
df_chats.head(10)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Did not pass numpy.dtype object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
Out[26]:

,ChatId,list_of_chats,lower_chats,summarized_text,stopwords_removed,most_freq_removed,cleaned_text,text_lemmatized
0,54137,Hi I spoke to Stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rec...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,speak stella earlier regard recent gourmet 202...
1,54141,Hello. This is my 3rd time on the chat I want ...,hello. this is my 3rd time on the chat i want ...,this is my 3rd time on the chat i want to cha...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnect e...
2,54143,"Hello there, Do i get repeated offers from sam...","hello there, do i get repeated offers from sam...",,,,,
3,54144,مرحبًا بك معنا مجددًا كان لديك 770 ابتسامة وا...,مرحبًا بك معنا مجددًا كان لديك 770 ابتسامة وان...,هل هناك أي شيء آخر يمكنني مساعدتك به ؟ [trans...,هل هناك أي شيء آخر يمكنني مساعدتك به transferc...,هل هناك أي شيء آخر يمكنني مساعدتك به transferc...,هل هناك أي شيء آخر يمكنني مساعدتك به transferc...,هل هناك أي شيء آخر يمكنني مساعدتك به transferc...
4,54145,Hi I was chatting with Greta How do I call you...,hi i was chatting with greta how do i call you...,hi i was chatting with greta how do i call yo...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chat greta call might easier purchase 2021 bun...
5,54147,Hi I cannot see any offer in Food buy 1 get 1 ...,hi i cannot see any offer in food buy 1 get 1 ...,,,,,
6,54148,"Hello, I bought Entertainer dubai classic yest...","hello, i bought entertainer dubai classic yest...",jeeyeon an/ ajycom1120@gmail.com and my order...,jeeyeon ajycom1120gmailcom order number 101387...,jeeyeon an ajycom1120gmailcom order number 101...,jeeyeon an ajycom1120gmailcom order number 101...,jeeyeon ajycom1120gmailcom order number 101387...


In [0]:
df_chats['lower_chats'].values[1]

Out[14]: 'hello. this is my 3rd time on the chat i want to change my entertainer package i have been disconnected every single time i’ve tried to ask so far i’ve been told to wait and everytime i’ve waited i have been disconnected by the support person'

In [0]:
df_chats['summarized_text'].values[1]

Out[15]: ' this is my 3rd time on the chat i want to change my entertainer package i have been disconnected every single time i’ve tried to ask so far i’ve been told to wait and everytime i’ve waited i have been disconnected by the support person'

In [0]:
df_chats['lower_chats'].values[2]

Out[16]: 'hello there, do i get repeated offers from same restaurant or is only 3 coupons max?'

In [0]:
df_chats['summarized_text'].values[2]

Out[17]: ''

In [0]:
df_chats['lower_chats'].values[3]

Out[18]: 'مرحبًا بك معنا مجددًا كان لديك 770 ابتسامة وانتهت للأسف لأنك لم تجددي اشتراكك قبل انتهاء العام . هل هناك أي شيء آخر يمكنني مساعدتك به ؟ [transferchat]_transferred by leslie to amged (عربي (ar)) (la) مرحبا كان لدي سمايلز واختفت فجاة لماذا ؟ البريد ra11122211@gmail.com الاسم roz abdullah تم ارتباط الحساب ولكن بدون سمايلز ليه ؟ ليه انتهت ؟ اوك تمام شكرا'

In [0]:
df_chats['summarized_text'].values[3]

Out[19]: ' هل هناك أي شيء آخر يمكنني مساعدتك به ؟ [transferchat]_transferred by leslie to amged (عربي (ar)) (la) مرحبا كان لدي سمايلز واختفت فجاة لماذا ؟ البريد ra11122211@gmail.com الاسم roz abdullah تم ارتباط الحساب ولكن بدون سمايلز ليه ؟ ليه انتهت ؟ اوك تمام شكرا'

In [0]:
df_chats['lower_chats'].values[4]

Out[20]: 'hi i was chatting with greta how do i call you might just be easier my purchase for 2021 bundle doesn’t show i need to confirm the purchase and for the third time my name is harshika daryanani my email address is daryananiharshika@gmail.com [nickchange] 0585504142 no i purchased the 2021 bundle i already had / have the 2020 key ? and i bought the 2021 in october , but why is it showing as 2020 bundle when i already was using the 2020 one for the whole year ? doesn’t make sense please advice , your chat keeps getting disconnected and i have to keep repeating myself ????????'

In [0]:
df_chats['summarized_text'].values[4]

Out[21]: ' hi i was chatting with greta how do i call you might just be easier my purchase for 2021 bundle doesn’t show i need to confirm the purchase and for the third time my name is harshika daryanani my email address is daryananiharshika@gmail.com [nickchange] 0585504142 no i purchased the 2021 bundle i already had / have the 2020 key ?'

In [0]:
# # SECOND TRY
# df_chats = prep_chat_data()
# df_chats.head(10)

In [0]:
# Drop any rows with feature col empty
df_chats=df_chats.drop(df_chats.index[[2,3,5]])
df_chats.reset_index(drop=True,inplace = True)
df_chats.head()

Out[27]:

,ChatId,list_of_chats,lower_chats,summarized_text,stopwords_removed,most_freq_removed,cleaned_text,text_lemmatized
0,54137,Hi I spoke to Stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rec...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,speak stella earlier regard recent gourmet 202...
1,54141,Hello. This is my 3rd time on the chat I want ...,hello. this is my 3rd time on the chat i want ...,this is my 3rd time on the chat i want to cha...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnect e...
2,54145,Hi I was chatting with Greta How do I call you...,hi i was chatting with greta how do i call you...,hi i was chatting with greta how do i call yo...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chat greta call might easier purchase 2021 bun...
3,54148,"Hello, I bought Entertainer dubai classic yest...","hello, i bought entertainer dubai classic yest...",jeeyeon an/ ajycom1120@gmail.com and my order...,jeeyeon ajycom1120gmailcom order number 101387...,jeeyeon an ajycom1120gmailcom order number 101...,jeeyeon an ajycom1120gmailcom order number 101...,jeeyeon ajycom1120gmailcom order number 101387...


In [0]:
df_quest['cleaned_text'] = df_quest['cleaned_text'].str.replace('[^\w\s]','')

In [0]:
df_quest.head()

Out[29]:

,Categroy,Questions,cleaned_text,stopwords_removed,text_lemmatized
0,14 day trial,Can I try before I buy?,can i try before i buy,try buy,try buy
1,14 day trial,How can I try the offers?,how can i try the offers,try offers,try offer
2,14 day trial,How can I try the app?,how can i try the app,try app,try app
3,14 day trial,Can I try first?,can i try first,try first,try first
4,14 day trial,Can I try an offer before I buy?,can i try an offer before i buy,try offer buy,try offer buy


In [0]:
# COUNT OF QUESTIONS IN EACH CATEGORIES
df_quest.groupby(['Categroy'], sort = False).count() 

Out[30]:

,Questions,cleaned_text,stopwords_removed,text_lemmatized
Categroy,,,,
14 day trial,33,33,33,33
General App Questions,100,100,100,100
Cheers,17,17,17,17
Account Questions,41,41,41,41
Complaints,12,12,12,12
Family Accounts,134,134,134,134
General,70,70,70,70
Getaways,67,67,67,67
Monthly Offers,11,11,11,11


In [0]:
# KEEP FIRST ENTRY FROM EACH GROUP FOR TESTING
df_test=df_quest.groupby('Categroy').first().reset_index()
df_test.head()

Out[31]:

,Categroy,Questions,cleaned_text,stopwords_removed,text_lemmatized
0,14 day trial,Can I try before I buy?,can i try before i buy,try buy,try buy
1,25% Offers UAE,How do I use my 25% off offers?,how do i use my 25 off offers,use 25 offers,use 25 offer
2,Account Questions,"I need to change my email address, please advi...",i need to change my email address please advi...,need change email address please advise,need change email address please advise
3,Adrenaline,How do I access Adrenaline offers?,how do i access adrenaline offers,access adrenaline offers,access adrenaline offer
4,Cheers,How do I access my Cheers offers?,how do i access my cheers offers,access cheers offers,access cheer offer


In [0]:
from sklearn.preprocessing import LabelEncoder
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df_test['labels'] = labelencoder.fit_transform(df_test['Categroy'])
df_test.head()

Out[44]:

,Categroy,Questions,cleaned_text,stopwords_removed,text_lemmatized,labels
0,14 day trial,Can I try before I buy?,can i try before i buy,try buy,try buy,0
1,25% Offers UAE,How do I use my 25% off offers?,how do i use my 25 off offers,use 25 offers,use 25 offer,1
2,Account Questions,"I need to change my email address, please advi...",i need to change my email address please advi...,need change email address please advise,need change email address please advise,2
3,Adrenaline,How do I access Adrenaline offers?,how do i access adrenaline offers,access adrenaline offers,access adrenaline offer,3
4,Cheers,How do I access my Cheers offers?,how do i access my cheers offers,access cheers offers,access cheer offer,4


In [0]:
# DROP FIRST ENTRY FROM EACH GROUP AND KEEP THE REST FOR TRAINING
df_train=df_quest.drop(df_quest.groupby(['Categroy']).head(1).index, axis=0)
df_train.head()

Out[32]:

,Categroy,Questions,cleaned_text,stopwords_removed,text_lemmatized
1,14 day trial,How can I try the offers?,how can i try the offers,try offers,try offer
2,14 day trial,How can I try the app?,how can i try the app,try app,try app
3,14 day trial,Can I try first?,can i try first,try first,try first
4,14 day trial,Can I try an offer before I buy?,can i try an offer before i buy,try offer buy,try offer buy
5,14 day trial,I want a free trial,i want a free trial,want free trial,want free trial


In [0]:
from sklearn.preprocessing import LabelEncoder
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df_train['labels'] = labelencoder.fit_transform(df_train['Categroy'])
df_train.head()

Out[45]:

,Categroy,Questions,cleaned_text,stopwords_removed,text_lemmatized,labels
1,14 day trial,How can I try the offers?,how can i try the offers,try offers,try offer,0
2,14 day trial,How can I try the app?,how can i try the app,try app,try app,0
3,14 day trial,Can I try first?,can i try first,try first,try first,0
4,14 day trial,Can I try an offer before I buy?,can i try an offer before i buy,try offer buy,try offer buy,0
5,14 day trial,I want a free trial,i want a free trial,want free trial,want free trial,0


In [0]:
# Train & Test subsets
X_train, y_train = df_train.iloc[:, 2].values, df_train.iloc[:, 0].values.reshape(-1, 1)
X_test, y_test = df_test.iloc[:, 2].values, df_test.iloc[:, 0].values.reshape(-1, 1)

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer


# Transform each text into a vector of word counts
vectorizer = CountVectorizer(stop_words="english",
                             preprocessor=clean_text)

training_features = vectorizer.fit_transform(df_train['text_lemmatized'])    
test_features = vectorizer.transform(df_test['text_lemmatized'])

# Training
model = LinearSVC()
model.fit(training_features, df_train['labels'])
y_pred = model.predict(test_features)

# Evaluation
acc = accuracy_score(df_test['labels'], y_pred)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

Accuracy on the IMDB dataset: 100.00

In [0]:
test_features = vectorizer.transform(df_chats['text_lemmatized'])
y_pred = model.predict(test_features)
y_pred

Out[52]: array([17, 0, 17, 15])

In [0]:
labelencoder.inverse_transform(y_pred)

Out[61]: array(['Pre Purchase', '14 day trial', 'Pre Purchase', 'Orders'],
 dtype=object)

In [0]:
df_chats['prediction']=labelencoder.inverse_transform(y_pred)

In [0]:
df_chats.head()

Out[64]:

,ChatId,list_of_chats,lower_chats,summarized_text,stopwords_removed,most_freq_removed,cleaned_text,text_lemmatized,prediction
0,54137,Hi I spoke to Stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rec...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,speak stella earlier regard recent gourmet 202...,Pre Purchase
1,54141,Hello. This is my 3rd time on the chat I want ...,hello. this is my 3rd time on the chat i want ...,this is my 3rd time on the chat i want to cha...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnect e...,14 day trial
2,54145,Hi I was chatting with Greta How do I call you...,hi i was chatting with greta how do i call you...,hi i was chatting with greta how do i call yo...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chat greta call might easier purchase 2021 bun...,Pre Purchase
3,54148,"Hello, I bought Entertainer dubai classic yest...","hello, i bought entertainer dubai classic yest...",jeeyeon an/ ajycom1120@gmail.com and my order...,jeeyeon ajycom1120gmailcom order number 101387...,jeeyeon an ajycom1120gmailcom order number 101...,jeeyeon an ajycom1120gmailcom order number 101...,jeeyeon ajycom1120gmailcom order number 101387...,Orders


In [0]:
df_chats['list_of_chats'].values[0] # Pre Purchase

Out[56]: "Hi I spoke to Stella earlier regarding my recent Gourmet 2021 purchase I want to change my purchase from Gourmet 2021 to Classic. Some of the restaurants from Gourmet 2020 is not in 2021 anymore and the listed restaurants are very less now. Okay I want refund Gourmet 2021 doesn’t have much restaurants now compared to 2020 and the restaurants that I like is not there anymore sandrheasacares@gmail.com Sandrhea Sacares [NICKCHANGE] Sure Yes,I did. Disappointed as the restaurants listed in Gourmet 2020 is not there in 2021 anymore Yes. How long will it take to reflect in the bank? Can I know why some of the restaurants from Gourmet 2020 is now available in Classic 2021? No worries. I can wait. I just don’t think the Gourmet 2021 is a good value for money anymore 0565654631 Hi! How may I help you? Exchange of product is not possible. Let me see what I can do for you. Please share your email address. Thanks and your name please? Good to have you here, Sandrhea Allow me a moment please. I see you bought Dubai Gourmet 2021 today. Are you looking to request a refund? Just to let you know, refunds tend to reach customer's account within 10 working days. Alright, can I have your phone number please? Thanks. We usually contact back within 1-2 business days maximum. However, on weekends it takes around 72 hours. Would there be anything else?"

In [0]:
df_chats['list_of_chats'].values[1] # 14 day trial

Out[57]: 'Hello. This is my 3rd time on the chat I want to change my entertainer package I have been disconnected every single time I’ve tried to ask so far I’ve been told to wait and everytime I’ve waited I have been disconnected by the support person'

In [0]:
df_chats['list_of_chats'].values[2] # Pre Purchase

Out[58]: 'Hi I was chatting with Greta How do I call you might just be easier My purchase for 2021 bundle doesn’t show I need to confirm the purchase and for the third time my name is harshika daryanani my email address is daryananiharshika@gmail.com [NICKCHANGE] 0585504142 No I purchased the 2021 bundle I already had / have the 2020 Key ? And I bought the 2021 in October , but why is it showing as 2020 bundle when I already was using the 2020 one for the whole year ? Doesn’t make sense Please advice , your chat keeps getting disconnected and I have to keep repeating myself ????????'

In [0]:
df_chats['list_of_chats'].values[3] # Order

Out[60]: "Hello, I bought Entertainer dubai classic yesterday. But I can't find my new voucher at all. Jeeyeon An/ ajycom1120@gmail.com And my order number 101387510-81861 [NICKCHANGE]"

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


# Transform each text into a vector of word counts
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=clean_text,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(df_train['text_lemmatized'])    
test_features = vectorizer.transform(df_test['text_lemmatized'])

# Training
model = LinearSVC()
model.fit(training_features, df_train["labels"])
y_pred = model.predict(test_features)

# Evaluation
acc = accuracy_score(df_test["labels"], y_pred)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

Accuracy on the IMDB dataset: 91.30

In [0]:
test_features = vectorizer.transform(df_chats['text_lemmatized'])
y_pred = model.predict(test_features)
y_pred

Out[66]: array([17, 0, 17, 6])

In [0]:
df_chats['prediction']=labelencoder.inverse_transform(y_pred)

In [0]:
df_chats.head()

Out[68]:

,ChatId,list_of_chats,lower_chats,summarized_text,stopwords_removed,most_freq_removed,cleaned_text,text_lemmatized,prediction
0,54137,Hi I spoke to Stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rece...,hi i spoke to stella earlier regarding my rec...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,spoke stella earlier regarding recent gourmet ...,speak stella earlier regard recent gourmet 202...,Pre Purchase
1,54141,Hello. This is my 3rd time on the chat I want ...,hello. this is my 3rd time on the chat i want ...,this is my 3rd time on the chat i want to cha...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnected...,3rd time chat want change package disconnect e...,14 day trial
2,54145,Hi I was chatting with Greta How do I call you...,hi i was chatting with greta how do i call you...,hi i was chatting with greta how do i call yo...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chatting greta call might easier purchase 2021...,chat greta call might easier purchase 2021 bun...,Pre Purchase
3,54148,"Hello, I bought Entertainer dubai classic yest...","hello, i bought entertainer dubai classic yest...",jeeyeon an/ ajycom1120@gmail.com and my order...,jeeyeon ajycom1120gmailcom order number 101387...,jeeyeon an ajycom1120gmailcom order number 101...,jeeyeon an ajycom1120gmailcom order number 101...,jeeyeon ajycom1120gmailcom order number 101387...,"Delivery Dubai, Abu Dhabi, Qatar"
